In [1]:
import os
import torch
from torch.utils.data import TensorDataset, DataLoader
import torchvision
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

**The test numpy files and the model files should be in the same directory as this notebook.**

In [2]:
# Assign and describe resources
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print('GPU: ', torch.cuda.get_device_name(device=device))
core_count = len(os.sched_getaffinity(0))
print('CPU cores: ', core_count)

# current working directory
wd = os.getcwd()
print('Working directory: ', wd)

# Loading Data
labels_train = np.load(wd+'/nodup_labels.npy')
data_train = np.load(wd+'/nodup_data.npy')

# scale and reshape data
reshaped_data = data_train.reshape((300,300,3,data_train.shape[1]))
reshaped_data = np.moveaxis(reshaped_data, 
                            source=[0, 1, 2, 3], 
                            destination=[2, 3, 1, 0])

x_te = torch.Tensor(reshaped_data)
labels_train = np.array(labels_train,dtype=int)
t_te = torch.Tensor(labels_train)
max_val = x_te.max()
if max_val == 255:
    x_te = x_te/255
    
# Prepare data for pytorch
image_dataset = TensorDataset(x_te,t_te)
dataloader = torch.utils.data.DataLoader(image_dataset, 
                                          batch_size=1,
                                          shuffle=True, 
                                          num_workers=core_count)

# Text labels
class_names = ['Stop','Yield','Red Light','Green Light','Roundabout','Right Turn Only',
                'Do Not Enter','Crosswalk','Handicap Parking','No Parking']

# Data specifications
dataset_size = len(image_dataset)
class_num = len(class_names)

print('Number of known classes: ', class_num)
print('Number of images: ', dataset_size)

GPU:  NVIDIA A100-SXM4-80GB
CPU cores:  2
Working directory:  /blue/eee4773/gmarais/Transfer_learning
Number of known classes:  10
Number of images:  6130


In [ ]:
# make testing statistics
    # ROC curve
    # confusion matrix
    # class accuracy

In [ ]:
# Import model structure and define number of classes
class_num = 10
model = torchvision.models.efficientnet_v2_l(weights='EfficientNet_V2_L_Weights.IMAGENET1K_V1')
num_ftrs = model.classifier[1].in_features
model.classifier = torch.nn.Linear(num_ftrs, class_num)

In [ ]:
model.load_state_dict(torch.load(wd+'/best_model_state.pt'))
model.to(device)
model.eval()
model.train(mode=False)

In [ ]:
with torch.no_grad():
    for i, (inputs, labels) in enumerate(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        labels = labels.to(int)
        
        outputs = model(inputs)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        preds = (probabilities >= 0.0009)[0].nonzero()#.cpu().flatten().tolist()
        print(outputs)
        print(probabilities)
        print(preds, '\n\n')

In [ ]:
# apply model to data 
# calculate accuracy metrics for model application (make sure accuracies are reliable)

# use softmax to give probs  (maybe necessary in training)
# use threshold to apply non class